# MNIST With PyTorch Training
This program runs in about 52.337 seconds (Windows 11, i7, 16 GB)

## Prerequisites
Install the following packages

In [ ]:
try:
    from cloudmesh.common.StopWatch import StopWatch
except:  # noqa: E722
    ! pip install cloudmesh-common
    from cloudmesh.common.StopWatch import StopWatch

## Import Libraries

In [1]:
StopWatch.start('total')
StopWatch.start('import')
StopWatch.progress(0)

import torch    # noqa: E402
from torchvision import datasets, transforms    # noqa: E402
from torch import nn    # noqa: E402
from torch import optim    # noqa: E402

StopWatch.stop('import')
StopWatch.progress(2)

# cloudmesh status=running progress=0 pid=12404


C:\Users\abeck\ENV3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# cloudmesh status=running progress=2 pid=12404


## Pre-Process Data

Here we download the data using PyTorch data utils and transform the data by using a normalization function. PyTorch provides a data loader abstraction called a `DataLoader` where we can set the batch size, data shuffle per batch loading. Each data loader expecte a Pytorch Dataset. The DataSet abstraction and DataLoader usage can be found [here](https://pytorch.org/tutorials/recipes/recipes/loading_data_recipe.html) 

In [2]:
StopWatch.start("preprocess")

# Data transformation function
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# DataSet
train_data_set = datasets.MNIST('drive/My Drive/mnist/data/', download=True, train=True, transform=transform)
validation_data_set = datasets.MNIST('drive/My Drive/mnist/data/', download=True, train=False, transform=transform)


train_loader = torch.utils.data.DataLoader(train_data_set, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data_set, batch_size=32, shuffle=True)

StopWatch.stop("preprocess")
StopWatch.progress(3)

100%|██████████| 9912422/9912422 [00:01<00:00, 5707423.76it/s]


Extracting drive/My Drive/mnist/data/MNIST\raw\train-images-idx3-ubyte.gz to drive/My Drive/mnist/data/MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28828104.19it/s]


Extracting drive/My Drive/mnist/data/MNIST\raw\train-labels-idx1-ubyte.gz to drive/My Drive/mnist/data/MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5215801.05it/s]


Extracting drive/My Drive/mnist/data/MNIST\raw\t10k-images-idx3-ubyte.gz to drive/My Drive/mnist/data/MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 7304650.60it/s]


Extracting drive/My Drive/mnist/data/MNIST\raw\t10k-labels-idx1-ubyte.gz to drive/My Drive/mnist/data/MNIST\raw

# cloudmesh status=running progress=3 pid=12404


## Define Network

Here we select the matching input size compared to the network definition. 
Here data reshaping or layer reshaping must be done to match input data shape with the network input shape. Also we define a set of hidden unit sizes along with the output layers size. The `output_size` must match with the number of labels associated with the classification problem. The hidden units can be chosesn depending on the problem. `nn.Sequential` is one way to create the network. Here we stack a set of linear layers along with a softmax layer for the classification as the output layer. 

In [3]:
StopWatch.start("network")

input_size = 784
hidden_sizes = [128, 128, 64, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[2], hidden_sizes[3]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[3], output_size),
                      nn.LogSoftmax(dim=1))

                      
print(model)

StopWatch.stop("network")
StopWatch.progress(4)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=10, bias=True)
  (9): LogSoftmax(dim=1)
)
# cloudmesh status=running progress=4 pid=12404


## Define Loss Function and Optimizer

Read more about [Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions) and [Optimizers](https://pytorch.org/docs/stable/optim.html) supported by PyTorch.





In [4]:
StopWatch.start('loss')

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

StopWatch.stop('loss')
StopWatch.progress(5)

# cloudmesh status=running progress=5 pid=12404


# Model Training

In [5]:
StopWatch.start('train')

epochs = 5

for epoch in range(epochs):
    loss_per_epoch = 0
    for images, labels in train_loader:
        images = images.view(images.shape[0], -1)
    
        # Gradients cleared per batch
        optimizer.zero_grad()
        
        # Pass input to the model
        output = model(images)
        # Calculate loss after training compared to labels
        loss = criterion(output, labels)
        
        # backpropagation 
        loss.backward()
        
        # optimizer step to update the weights
        optimizer.step()
        
        loss_per_epoch += loss.item()
    average_loss = loss_per_epoch / len(train_loader)
    print("Epoch {} - Training loss: {}".format(epoch, average_loss))

StopWatch.stop('train')
StopWatch.progress(97)

Epoch 0 - Training loss: 0.806366236380736
Epoch 1 - Training loss: 0.22268100586583217
Epoch 2 - Training loss: 0.15054172303676605
Epoch 3 - Training loss: 0.12087311126689117
Epoch 4 - Training loss: 0.10063299378032485
# cloudmesh status=running progress=97 pid=12404


## Model Evaluation

Similar to training data loader, we use the validation loader to load batch by batch and run the feed-forward network to get the expected prediction and compared to the label associated with the data point. 

In [6]:
StopWatch.start('evaluation')

correct_predictions, all_count = 0, 0
# enumerate data from the data validation loader (loads a batch at a time)
for batch_id, (images,labels) in enumerate(validation_loader):
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    # at prediction stage, only feed-forward calculation is required. 
    with torch.no_grad():
        logps = model(img)

    # Output layer of the network uses a LogSoftMax layer
    # Hence the probability must be calculated with the exponential values. 
    # The final layer returns an array of probabilities for each label
    # Pick the maximum probability and the corresponding index
    # The corresponding index is the predicted label 
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_predictions += 1
    all_count += 1

print(f"Model Accuracy {(correct_predictions/all_count) * 100} %")

StopWatch.stop('evaluation')
StopWatch.stop('total')
StopWatch.benchmark()
StopWatch.progress(100)

Model Accuracy 96.3 %

+------------------+----------------------------------------------------------------------------------+
| Attribute        | Value                                                                            |
|------------------+----------------------------------------------------------------------------------|
| cpu              |                                                                                  |
| cpu_cores        | 4                                                                                |
| cpu_count        | 8                                                                                |
| cpu_threads      | 8                                                                                |
| date             | 2022-07-15 13:30:56.604028                                                       |
| frequency        | scpufreq(current=2803.0, min=0.0, max=2803.0)                                    |
| mem.available    | 6.2 GiB             

### Reference: 

1. [Torch NN Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
2. [Handwritten Digit Recognition Using PyTorch — Intro To Neural Networks](https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627)
3. [MNIST Handwritten Digit Recognition in PyTorch](https://nextjournal.com/gkoehler/pytorch-mnist)
